# LangChain para INVIAS

## 🔒 1. Instalación de librerías

### 🔓  Solo ejecutar este script si no se han instalado los paquetes para desarrollar el código

In [ ]:
""" 
Lista de los paquetes a instalar:

    upgrade:
        pip
        setuptools
        wheel
    Packages:
        langchain 
        pypdf 
        openai 
        chromadb 
        tiktoken
        langchain-community
"""
import subprocess

comandos = [
    ["pip", "install", "--upgrade", "pip", "setuptools", "wheel"],
    ["pip", "install", "langchain", "pypdf", "openai", "chromadb", "tiktoken"],
    ["pip", "install", "-U", "langchain-community"],
    ["python", "-m", "pip", "install", "--upgrade", "pip"]
]

log_path = "instalacion_log.txt"

with open(log_path, "w", encoding="utf-8") as log_file:
    for i, cmd in enumerate(comandos, start=1):
        log_file.write(f"\n🔧 Ejecutando comando {i}: {' '.join(cmd)}\n")
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        log_file.write("✅ STDOUT:\n")
        log_file.write(result.stdout + "\n")
        
        if result.stderr:
            log_file.write("⚠️ STDERR:\n")
            log_file.write(result.stderr + "\n")

print(f"✅ Resultado guardado en {log_path}")


## 🟢 2. Configuración de API Key de OpenAI

In [1]:
import os
from openai import OpenAI
# Recuperar la clave API de la variable de entorno
api_key_environ = os.environ.get("OPENAI_API_KEY")
 
# Verificar que la clave API esté disponible
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada o está vacía.")
 
# Inicializar el cliente de OpenAI con la clave API
client = OpenAI(api_key=api_key_environ)
 
# Usar el cliente para tus tareas
print("¡Cliente de OpenAI inicializado correctamente!")

¡Cliente de OpenAI inicializado correctamente!


## 🔒 3. Carga de documents

### 🔓  Solo ejecutar este script si no se ha hecho el proceso de embedding

In [ ]:
import requests
from langchain.document_loaders import PyPDFLoader
import os

relative_pdf_path = "../../../assets/DG_docs/PDFs_test/"

ml_papers = []

for i, file_name in enumerate(os.listdir(relative_pdf_path)):
    if file_name.lower().endswith(".pdf"):
        full_pdf_path = os.path.join(relative_pdf_path,file_name)
        print(f"📄 Cargando {file_name}")

        loader = PyPDFLoader(full_pdf_path)
        data = loader.load() # AI_Queries\code_explanation\ai_query-langc_v01-PyPDFLoader(filename).loader.load()_usage.md
        ml_papers.extend(data) # AI_Queries\code_explanation\ai_query-langc_v01-.extend_usage.md
        # print (ml_papers) # AI_Queries/code_explanation/ai_query-langc_v01-list_start_end_usage.md
# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Esto es todo el contenido de `ml_papers:`')
print(f"""
🆗 Todos los documentos estan cargados en ml_papers.
➖ Total de fragmentos: {len(ml_papers)}
➖ Los fragmentos son cada una de las hojas de cada uno de los {len(os.listdir(relative_pdf_path))} archivos en la carpeta {relative_pdf_path}
➖ Este script se ejecuta desde {os.getcwd()}
➖ Este es el contenido de la última hoja cargada {ml_papers[-1]}
""")

## 🔒 4. Split de documents

### 🔓 Solo ejecutar este script para desarrollar el proceso de embedding; este script depende del script anterior 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    # AI_Queries\code_explanation\ai_query-langc_v01-chunk_usage.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-max_tokens_Chatgptmodels.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-meaning_inputpromptandanswer.md 
    # AI_Queries\code_explanation\ai_query-langc_v01-retrieval_meaning.md
    
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [ ]:
len(documents), documents[0]

## 🔒 5. Embeddings e ingesta a base de datos vectorial 

⚠️ advertencia de uso de esta sección ⚠️

### 🔓 5.1. Solo ejecutar este script para desarrolla el proceso de embedding; este script depende del script anterior

**Aquí se consume recurso de la API de OpenAI**

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# AI_Queries\code_explanation\ai_query-langc_v01-Embeddings_and_Vector_Store_Ingestion.md

# 1. Crear embeddings con el modelo oficial de OpenAI
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # ⚠️ cambiar a "text-embedding-3-small"

# 2. Definir carpeta para almacenar la base de datos vectorial
persist_directory = "chroma_db" #

# 3. Crear la base desde documentos y embeddings
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

# 4. Guardar la base en disco
vectorstore.persist()
print("✅ Base de datos Chroma guardada en:", persist_directory)


# 5. Cargar la base vectorial guardada en disco
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db"
)

# 6. Usar como retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

### 5.2. 🟢

Fue necesario crear copia de las lineas "*# 1. Crear embeddings con el modelo oficial de OpenAI*", "*# 5. Cargar la base vectorial guardada en disco*" y "*# 6. Usar como retriever*" para **solamente hacer uso de la Based de Datos de embeddings `chroma_db` ya creada y evitar recalcularla**.

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 1. Modelo de embeddings (debe ser el mismo usado al crear la base)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # ⚠️ cambiar a "text-embedding-3-small"

# 5. Cargar la base vectorial guardada en disco
vectorstore = Chroma(
    embedding_function=embeddings,
    persist_directory="chroma_db"
)

# 6. Usar como retriever
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

C:\Users\mmartinezo\AppData\Local\Temp\ipykernel_19756\1269346108.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # ⚠️ cambiar a "text-embedding-3-small"
C:\Users\mmartinezo\AppData\Local\Temp\ipykernel_19756\1269346108.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


## 6. Modelos de chat y cadenas para consulta de información

### 🚩 6.1. Recuperación sin trazabilidad – Primer acercamiento con LangChain; ~~No cita las fuentes de donde extrae la información~~.

In [ ]:
#AI_Queries/code_explanation/ai_query-langc_v01-Chat_Models_and_Retrieval_Chains_for_Information_Querying.md

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

chat = ChatOpenAI(
    openai_api_key=api_key_environ,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# AI_Queries/code_explanation/ai_query-langc_v01-RetrievalQA.from_chain_type_usage.md
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
# AI_Queries/code_explanation/ai_query-langc_v01-qa_chain.run()_usage.md

query = "qué es CCPT?"
qa_chain.run(query)

🧠 Comparación entre Script 1 y Script 2

| Aspecto                           | **Script anterior**                                 | **Script siguiente**                                                       |
| --------------------------------- | --------------------------------------------------- | -------------------------------------------------------------------------- |
| **Importación del modelo**        | `from langchain.chat_models` (obsoleto)             | `from langchain_openai` (recomendado)                                      |
| **Versión del API**               | Antiguo LangChain monolítico                        | Nuevo ecosistema modular `langchain-openai`                                |
| **Método de ejecución**           | `qa_chain.run(query)` (⚠️ obsoleto)                 | `qa_chain.invoke({"query": query})` (✅ recomendado)                        |
| **Fuentes del resultado**         | ❌ No devuelve los documentos fuente                 | ✅ Incluye los documentos fuente con `return_source_documents=True`         |
| **Retorno estructurado**          | Sólo devuelve texto plano                           | Devuelve un diccionario con respuesta y `source_documents`                 |
| **Transparencia institucional**   | Baja (no se puede verificar origen de la respuesta) | Alta (permite validar en qué parte de los documentos se basa la respuesta) |
| **Uso pedagógico recomendado**    | Para introducir conceptos básicos de `RetrievalQA`  | Para enseñar buenas prácticas actuales en trazabilidad y uso del LLM       |
| **Estabilidad futura del código** | Baja (usa módulos y métodos deprecados)             | Alta (adaptado a la versión estable y mantenida de LangChain)              |

### (🚩Revisar por qué no está gerendo resultados🚩) 6.2. Con recuperación básica – El sistema responde y cita fuentes

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# Inicializa el modelo de lenguaje
chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    openai_api_key=api_key_environ
)

# Crear la cadena de preguntas y respuestas con fuentes
qa_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True  # ✅ Para obtener las fuentes
)

In [ ]:
query = " que pasa en Ocaña?"
result = qa_chain.invoke({"query": query})

# Imprimir respuesta
print("🧠 Respuesta:")
print(result['result'])

# Imprimir fuentes
print("\n📚 Fuentes:")
for i, doc in enumerate(result['source_documents']):
    print(f"\nFuente {i+1}:")
    print("Archivo:", doc.metadata.get("source", "desconocido"))
    print("Contenido:", doc.page_content[:400])  # Muestra primeros 400 caracteres


🧠 Comparación entre Script 1 y Script 2

| Aspecto                           | **Script anterior**                                 | **Script actual**                                                       |
| --------------------------------- | --------------------------------------------------- | -------------------------------------------------------------------------- |
| **Importación del modelo**        | `from langchain.chat_models` (obsoleto)             | `from langchain_openai` (recomendado)                                      |
| **Versión del API**               | Antiguo LangChain monolítico                        | Nuevo ecosistema modular `langchain-openai`                                |
| **Método de ejecución**           | `qa_chain.run(query)` (⚠️ obsoleto)                 | `qa_chain.invoke({"query": query})` (✅ recomendado)                        |
| **Fuentes del resultado**         | ❌ No devuelve los documentos fuente                 | ✅ Incluye los documentos fuente con `return_source_documents=True`         |
| **Retorno estructurado**          | Sólo devuelve texto plano                           | Devuelve un diccionario con respuesta y `source_documents`                 |
| **Transparencia institucional**   | Baja (no se puede verificar origen de la respuesta) | Alta (permite validar en qué parte de los documentos se basa la respuesta) |
| **Uso pedagógico recomendado**    | Para introducir conceptos básicos de `RetrievalQA`  | Para enseñar buenas prácticas actuales en trazabilidad y uso del LLM       |
| **Estabilidad futura del código** | Baja (usa módulos y métodos deprecados)             | Alta (adaptado a la versión estable y mantenida de LangChain)              |

🧠 Diferencias clave entre ambos scripts

| Característica       | Script con `create_retrieval_chain` (moderno, script siguiente)            | Script con `RetrievalQA.from_chain_type` (clásico, script anterior) |
| -------------------- | -------------------------------------------------------- | -------------------------------------------------- |
| API utilizada        | Moderna (modular, flexible)                              | Clásica (más sencilla, menos control)              |
| Cadena usada         | `create_retrieval_chain` + `combine_docs_chain`          | `RetrievalQA.from_chain_type`                      |
| Prompt               | Totalmente personalizado con `PromptTemplate`            | Interno, no configurable por defecto               |
| Control de flujo     | Separación clara de pasos (`retriever`, `prompt`, `LLM`) | Todo en una sola línea                             |
| Adaptabilidad futura | Alta (recomendado en versiones recientes)                | Limitada (deprecated en versiones nuevas)          |


### 6.3. Con recuperación moderna – El sistema responde y muestra fuentes

#### 🟢 6.3.0. Retriever

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_openai import ChatOpenAI

# 1. LLM moderno
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0, api_key=api_key_environ)

# 2. Prompt simple para combinar documentos
prompt = PromptTemplate.from_template(
    "Usa los siguientes documentos para responder la pregunta.\n\n{context}\n\nPregunta: {input}"
)

# 3. Cadena para combinar documentos (tipo "stuff")
combine_docs_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

# 4. Cadena de recuperación completa con el retriever
qa_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain
)


In [4]:
response = qa_chain.invoke({"input": " deme un listado de los documentos donde se menciona CCPT?"})

print("🧠 Respuesta:")
print(response["answer"])

print("\n📚 Documentos usados:")
for i, doc in enumerate(response["context"]):
    print(f"\nFuente {i+1}:", doc.metadata.get("source", "desconocido"))
    print(doc.page_content[:300])


🧠 Respuesta:
1. Convenios-Contratos suscritos CCPT
2. Anexo No. 1 - Convenios-Contratos suscritos CCPT
3. Consolidado CCPT Detallado
4. Convenios CCPT.xlsx

📚 Documentos usados:

Fuente 1: ../../../assets/DG_docs/PDF_test_gradio/2025S-VBOG-005929.pdf
que en el documento adjunto mencionado en la respuesta al numeral 1º de la petición y que se denomina “Convenios-Contratos suscritos CCPT” , se relaciona la totalidad de los convenios y contratos suscritos en el marco del programa Caminos Comunitarios de la Paz Total, especificando los convenios cel

Fuente 2: ../../../assets/DG_docs/PDF_test_gradio/2025S-VBOG-020627.pdf
En la parte final de la presente respuesta usted encontrará un enlace donde podrá evidenciar un archivo denominado “Anexo No. 1 - Convenios-Contratos suscritos CCPT” allí encontrará la relación de convenios y contratos suscritos en las vigencias 2023 – 2024 con la información del departamento, munic

Fuente 3: ../../../assets/DG_docs/PDF_test_gradio/2025S-VBOG-020627 - cam

🧠 Diferencias clave entre ambos scripts

| Característica       | Script con `create_retrieval_chain` (moderno)            | Script con `RetrievalQA.from_chain_type` (clásico) |
| -------------------- | -------------------------------------------------------- | -------------------------------------------------- |
| API utilizada        | Moderna (modular, flexible)                              | Clásica (más sencilla, menos control)              |
| Cadena usada         | `create_retrieval_chain` + `combine_docs_chain`          | `RetrievalQA.from_chain_type`                      |
| Prompt               | Totalmente personalizado con `PromptTemplate`            | Interno, no configurable por defecto               |
| Control de flujo     | Separación clara de pasos (`retriever`, `prompt`, `LLM`) | Todo en una sola línea                             |
| Adaptabilidad futura | Alta (recomendado en versiones recientes)                | Limitada (deprecated en versiones nuevas)          |


#### 🟢 6.3.1 Gradio para seccion 6.3.

In [5]:
import gradio as gr

# Función para manejar la entrada y salida de QA Chain
def ask_question(question: str):
    # Invoca la cadena con la pregunta del usuario
    response = qa_chain.invoke({"input": question})
    # Extrae la respuesta
    answer = response.get("answer", "")
    # Construye la lista de fuentes y fragmentos
    context_snippets = []
    for i, doc in enumerate(response.get("context", [])):
        source = doc.metadata.get("source", "desconocido")
        snippet = doc.page_content[:300].replace("\n", " ")  # primeras 300 chars
        context_snippets.append(f"Fuente {i+1}: {source}\n{snippet}")
    context_text = "\n\n".join(context_snippets)
    return answer, context_text

# Definición de la interfaz de Gradio
iface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Ingrese su pregunta aquí…"),
    outputs=[
        gr.Textbox(label="Respuesta"),
        gr.Textbox(label="Documentos usados")
    ],
    title="Interfaz QA con LangChain",
    description="Ingrese una consulta y obtenga la respuesta junto con las fuentes utilizadas."
)

# Lanzar la aplicación (en local o con share=True para exponerla)
iface.launch()
#iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### 6.3.2 ❌ Falló la busqueda ❌

Entendido. Tu sospecha es excelente y es el camino correcto para diagnosticar el problema en un sistema RAG (Búsqueda y Generación Aumentada). Si el sistema no encuentra la información correcta, el problema casi siempre está en la etapa de **recuperación (*retrieval*)**, no en el modelo de lenguaje que genera la respuesta final.

Vamos a crear un "test de recuperación" para verificar exactamente qué información está encontrando el sistema antes de intentar responder. Este test nos permitirá aislar el problema.

##### Objetivo del Test: Evaluar al `Retriever`

El `retriever` es el componente encargado de buscar en tu base de datos vectorial (ChromaDB) los fragmentos de texto más relevantes para la pregunta que haces. Si no le entrega los fragmentos correctos al modelo de lenguaje (LLM), es imposible que el LLM genere una buena respuesta.

Vamos a ejecutar el `retriever` de forma aislada para inspeccionar su "trabajo".

---

##### Paso 1: Escoge un Caso de Prueba Específico

Primero, necesitas un "caso de oro" que puedas verificar manualmente.

1.  **Busca en tus PDFs:** Abre uno de los documentos PDF que usaste para crear la base de datos. Encuentra un dato muy específico y único. Por ejemplo, el nombre de un proyecto, un número de contrato, una fecha clave o una frase técnica.
2.  **Define tu "Golden Pair":**
    * **Pregunta de prueba:** Formula una pregunta que *debería* encontrar ese dato. Ejemplo: `"¿Cuál es el número del contrato para el mantenimiento de la vía Ocaña-Cúcuta?"`
    * **Contenido esperado:** Anota la frase o las palabras clave que *obligatoriamente* deben estar en los fragmentos recuperados para poder responder la pregunta. Ejemplo: `"contrato de obra pública No. 2345-2023"`.

##### Paso 2: Código para Ejecutar el Test

He preparado una versión simplificada de tu script que se enfoca **únicamente en realizar este test**. No utilizaremos el LLM para generar respuestas, solo para cargar los embeddings y usar el `retriever`.

Copia y ejecuta el siguiente código. Asegúrate de modificar las variables de la sección "CONFIGURACIÓN DEL TEST".

```python
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "que pasa en Ocaña?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["Ocaña", "emergencia", "deslizamiento"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

```

##### ¿Cómo Interpretar los Resultados?

1.  **Si el veredicto es "ÉXITO"**: Significa que tu base de datos y tu `retriever` funcionan correctamente. El problema podría estar en el `prompt` que le das al LLM o en el propio LLM (aunque es menos probable).
2.  **Si el veredicto es "FALLO"**: Confirma tu sospecha. El problema está en la base de conocimiento. El sistema no es capaz de encontrar los fragmentos de texto relevantes.

##### ¿Qué Hacer si el Test Falla?

Si el test falla, aquí tienes las causas más probables y cómo solucionarlas:

1.  **Problema con la División de Texto (`chunking`)**:
    * **Causa**: El `chunk_size=1500` puede ser muy grande, y la información clave se "diluye" con texto irrelevante en el mismo fragmento. O puede ser muy pequeño y parte una frase importante.
    * **Solución**: Prueba a regenerar tu base de datos con un `chunk_size` más pequeño (ej. 1000) y un `chunk_overlap` mayor (ej. 250). Esto crea más fragmentos, pero más enfocados.

2.  **Problema con la Extracción de Texto del PDF**:
    * **Causa**: Algunos PDFs son en realidad imágenes de texto (escaneados). `PyPDFLoader` no puede leer texto de imágenes.
    * **Solución**: Verifica el contenido de tus documentos justo después de cargarlos (la variable `ml_papers` en tu script original). Si ves caracteres extraños o texto sin sentido, necesitas usar una herramienta de **OCR (Reconocimiento Óptico de Caracteres)** para extraer el texto de esos PDFs antes de pasarlos a LangChain.

3.  **Problema con el Modelo de Embeddings**:
    * **Causa**: `text-embedding-ada-002` es un modelo más antiguo. Puede que no esté capturando el significado semántico de tu pregunta con la suficiente precisión.
    * **Solución**: Regenera tu base de datos usando un modelo más moderno y potente como `"text-embedding-3-small"` (más barato y eficiente) o `"text-embedding-3-large"` (máxima calidad). Tu propio código ya tiene un comentario sugiriendo este cambio.

Ejecuta este test y, según el resultado, podremos enfocarnos en la solución correcta.

##### 6.3.2.1. Analisis del documento 73SentenciaTutela.pdf by GEMINI

Excelente. He analizado el documento `73SentenciaTutela.pdf` y he extraído varios datos específicos y únicos que nos servirán perfectamente para crear los "casos de oro" y probar la eficacia de tu sistema de recuperación de información.

A continuación, te presento 3 casos de prueba con diferentes niveles de especificidad. Podemos usar uno o todos para diagnosticar tu sistema.

---

###### Caso de Prueba 1: **Número de Radicación (Dato Único)**

Este es el dato más específico y fácil de verificar. Es ideal para una primera prueba.

* [cite_start]**Dato específico encontrado:** El número de radicación del proceso de tutela es `080013187005202500020-00`[cite: 1, 7, 15, 23, 29, 37, 47, 55, 64, 74, 86, 94, 101, 110, 116, 125, 132, 140, 145, 149, 154, 163, 171, 180, 187, 195, 206, 222, 231, 239, 255, 262, 269, 276, 286, 297, 305, 313, 323, 329].
* **Pregunta de prueba sugerida:** `¿Cuál es el número de radicación de la acción de tutela de Gersel Luis Pérez Altamiranda?`
* **Contenido esperado (palabras clave):** El `retriever` debe devolver fragmentos que contengan obligatoriamente la cadena `080013187005202500020-00`.

###### Caso de Prueba 2: **Nombre de Resolución (Dato Específico)**

Este caso prueba si el sistema puede asociar una acción (distribuir un cobro) con un acto administrativo concreto.

* [cite_start]**Dato específico encontrado:** La `Resolución No. 3856 de 26 de agosto de 2024` es la que distribuye el cobro de la Contribución Nacional de Valorización[cite: 9, 11].
* **Pregunta de prueba sugerida:** `¿Qué resolución distribuye el cobro de la Contribución Nacional de Valorización para el proyecto Circunvalar de la Prosperidad?`
* **Contenido esperado (palabras clave):** El `retriever` debería encontrar fragmentos que incluyan `Resolución No. 3856` y `distribuye el cobro`.

###### Caso de Prueba 3: **Concepto Jurídico y Condición (Dato Complejo)**

Este caso es más avanzado. Evalúa si el sistema puede encontrar información conceptual y sus condiciones, no solo datos literales.

* [cite_start]**Dato específico encontrado:** La Ley 1819 de 2016, en su artículo 253, excluye del cobro de valorización a "las tierras comunales de grupos étnicos, las tierras de resguardo y el patrimonio arqueológico y cultural de la nación, cuando su titularidad radique en una entidad de derecho público"[cite: 112, 127, 166, 235].
* **Pregunta de prueba sugerida:** `¿Bajo qué condición están excluidas del cobro de valorización las tierras de grupos étnicos?`
* **Contenido esperado (palabras clave):** El `retriever` debe recuperar fragmentos que contengan `excluidos`, `tierras comunales`, `grupos étnicos` y, crucialmente, `titularidad radique en una entidad de derecho público`.

---

**Siguiente Paso Sugerido:**

Ahora que tenemos estos casos, te propongo que elijas uno de ellos (recomiendo empezar por el **Caso de Prueba 1** por su simplicidad) y lo usemos en el script de test que te proporcioné anteriormente.

Solo necesitas actualizar la sección `CONFIGURACIÓN DEL TEST` en ese código con la "Pregunta de prueba" y el "Contenido esperado" del caso que elijas. Al ejecutarlo, veremos inmediatamente si tu sistema puede encontrar esta información básica en el PDF.

###### 🪛 PREGUNTA 1: ¿Cuál es el número de radicación de la acción de tutela de Gersel Luis Pérez Altamiranda?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Cuál es el número de radicación de la acción de tutela de Gersel Luis Pérez Altamiranda?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["080013187005202500020-00"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

###### 🪛 PREGUNTA 2: ¿Qué resolución distribuye el cobro de la Contribución Nacional de Valorización para el proyecto Circunvalar de la Prosperidad?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Qué resolución distribuye el cobro de la Contribución Nacional de Valorización para el proyecto Circunvalar de la Prosperidad?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["Resolución No. 3856", "distribuye el cobro"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

###### 🪛 PREGUNTA 3: ¿Bajo qué condición están excluidas del cobro de valorización las tierras de grupos étnicos?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Bajo qué condición están excluidas del cobro de valorización las tierras de grupos étnicos?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["excluidos", "tierras comunales", "grupos étnicos", "titularidad radique en una entidad de derecho público"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

##### 6.3.2.2. Diagnóstico Definitivo: Verifiquemos qué "lee" `PyPDFLoader`

Ok, este resultado es la pieza de información más importante que podríamos haber obtenido y confirma nuestras sospechas de la manera más clara posible.

El hecho de que los tres tests hayan fallado, en especial el del **número de radicación** (que es un dato literal y único), nos dice con un 99% de certeza que el problema no está en la configuración de la búsqueda (`chunk_size`), ni en el modelo de `embeddings`.

**El problema fundamental está en la extracción del texto del propio archivo PDF.**

Para usar una analogía: le hemos pedido a nuestro sistema que busque una aguja en un pajar, pero al revisar el pajar, descubrimos que nunca metimos la aguja dentro. El "texto" que tu sistema tiene almacenado probablemente está corrupto, incompleto o es inexistente.

###### Diagnóstico Definitivo: Verifiquemos qué "lee" `PyPDFLoader`

La causa más común de este fallo total es que el PDF (`73SentenciaTutela.pdf`) no es un documento de texto, sino un **conjunto de imágenes escaneadas**. `PyPDFLoader` no puede leer texto de imágenes, por lo que extrae caracteres sin sentido o simplemente cadenas vacías.

Vamos a hacer un último test, el más simple y revelador de todos. Ejecutemos un código que solo cargue el PDF y nos muestre exactamente qué texto está extrayendo.

**Código de Diagnóstico de Extracción:**

```python
from langchain_community.document_loaders import PyPDFLoader
import os

# --- CONFIGURACIÓN ---
# Asegúrate de que el PDF esté en la misma carpeta que este script,
# o proporciona la ruta completa.
PDF_PATH = "73SentenciaTutela.pdf" 

# --- LÓGICA DE DIAGNÓSTICO ---

print(f"📄 Intentando cargar el archivo: {PDF_PATH}")

if not os.path.exists(PDF_PATH):
    print(f"❌ ERROR: No se encontró el archivo en la ruta especificada.")
else:
    try:
        loader = PyPDFLoader(PDF_PATH)
        pages = loader.load()
        
        print(f"\n✅ Archivo cargado. El PDF tiene {len(pages)} páginas.")
        print("--- Mostrando el texto extraído de las primeras 3 páginas ---")
        
        for i in range(min(3, len(pages))):
            page_content = pages[i].page_content
            print(f"\n\n--- PÁGINA {i+1} ---")
            # Imprimimos solo los primeros 500 caracteres para no saturar la pantalla
            print(page_content[:500])
            if not page_content.strip():
                print(">>> ¡ALERTA! El contenido de esta página está vacío o solo contiene espacios.")

    except Exception as e:
        print(f"\n❌ Ocurrió un error al cargar o procesar el PDF: {e}")

```

###### ¿Qué Esperamos Ver?

Ejecuta el código anterior. El resultado nos dirá todo:

  * **Resultado Esperado (si todo estuviera bien):** Verías el texto real de la sentencia, claramente legible. Por ejemplo: `Rama Judicial Consejo Superior de la Judicatura...`.
  * **Resultado Incorrecto (El más probable):** Verás una salida con texto sin sentido, símbolos extraños, o peor aún, una salida completamente vacía (`''`). **Esta es mi principal sospecha.**

-----

###### La Solución: Usar OCR (Reconocimiento Óptico de Caracteres)

Si confirmamos que el texto extraído es basura o está vacío, la solución es procesar el PDF con una herramienta de **OCR**. Esto convierte las imágenes de texto en texto real que LangChain sí puede entender.

Aquí te explico cómo modificar tu flujo de trabajo para incluir OCR.

####### 1\. Instalar las librerías necesarias

Necesitarás instalar `pytesseract`, una librería de Python que interactúa con el motor de OCR de Google (Tesseract), y `pdf2image` para convertir las páginas del PDF en imágenes.

```bash
pip install pytesseract pdf2image Pillow
```

Además, debes **instalar el motor Tesseract de Google** en tu sistema. Las instrucciones varían según tu sistema operativo:

  * **Windows:** Descarga el instalador desde [este enlace oficial](https://www.google.com/search?q=https://github.com/UB-Mannheim/tesseract/wiki) y asegúrate de añadir la ruta de instalación al PATH de tu sistema.
  * **macOS:** `brew install tesseract`
  * **Linux (Debian/Ubuntu):** `sudo apt-get install tesseract-ocr`

####### 2\. Nuevo Proceso para Cargar Documentos con OCR

Ahora, en lugar de usar `PyPDFLoader`, usarías una función personalizada que primero hace OCR y luego crea los objetos `Document` que LangChain necesita.

Este sería el reemplazo para tu celda "Carga de documents":

```python
from pdf2image import convert_from_path
import pytesseract
from langchain.docstore.document import Document
import os

# --- CONFIGURACIÓN ---
PDF_PATH = "73SentenciaTutela.pdf" # Ruta a tu PDF

# Si instalaste Tesseract en una ubicación no estándar (Windows),
# podrías necesitar especificar la ruta aquí.
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# --- FUNCIÓN DE CARGA CON OCR ---
def load_pdf_with_ocr(file_path):
    """
    Carga un PDF, extrae texto de cada página usando OCR y devuelve
    una lista de objetos Document de LangChain.
    """
    print("🔄 Convirtiendo páginas de PDF a imágenes...")
    images = convert_from_path(file_path)
    
    documents_ocr = []
    print(f"🔍 Procesando {len(images)} páginas con OCR...")
    
    for i, image in enumerate(images):
        # Usar Tesseract para extraer texto de la imagen
        # 'spa' es para español
        text = pytesseract.image_to_string(image, lang='spa')
        
        # Crear un documento de LangChain por cada página
        doc = Document(
            page_content=text,
            metadata={
                "source": os.path.basename(file_path),
                "page": i + 1
            }
        )
        documents_ocr.append(doc)
    
    print("✅ Proceso de OCR completado.")
    return documents_ocr


# --- EJECUCIÓN ---
# Reemplaza la carga original de PyPDFLoader con esta función
ml_papers = load_pdf_with_ocr(PDF_PATH)

# A partir de aquí, el resto de tu script sigue igual.
# La variable `ml_papers` ahora contiene el texto extraído correctamente.
print(f"\nTotal de páginas procesadas: {len(ml_papers)}")
print("\nContenido de la primera página (extraído con OCR):")
print(ml_papers[0].page_content[:500])

# Ahora puedes pasar `ml_papers` al `text_splitter`
# text_splitter = RecursiveCharacterTextSplitter(...)
# documents = text_splitter.split_documents(ml_papers)
# ...y continuar con el resto de tu script.
```

Te recomiendo primero ejecutar el **código de diagnóstico**. Una vez que confirmes que la extracción de texto falla, implementa la **solución con OCR**. Este es el camino correcto para resolver el problema de raíz.

###### 🪛 Código de Diagnóstico de Extracción:

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import os

# --- CONFIGURACIÓN ---
# Asegúrate de que el PDF esté en la misma carpeta que este script,
# o proporciona la ruta completa.
PDF_PATH = "../../../assets/DG_docs/PDF_test_gradio/73SentenciaTutela.pdf" 

# --- LÓGICA DE DIAGNÓSTICO ---

print(f"📄 Intentando cargar el archivo: {PDF_PATH}")

if not os.path.exists(PDF_PATH):
    print(f"❌ ERROR: No se encontró el archivo en la ruta especificada.")
else:
    try:
        loader = PyPDFLoader(PDF_PATH)
        pages = loader.load()
        
        print(f"\n✅ Archivo cargado. El PDF tiene {len(pages)} páginas.")
        print("--- Mostrando el texto extraído de las primeras 3 páginas ---")
        
        for i in range(min(3, len(pages))):
            page_content = pages[i].page_content
            print(f"\n\n--- PÁGINA {i+1} ---")
            # Imprimimos solo los primeros 500 caracteres para no saturar la pantalla
            print(page_content[:500])
            if not page_content.strip():
                print(">>> ¡ALERTA! El contenido de esta página está vacío o solo contiene espacios.")

    except Exception as e:
        print(f"\n❌ Ocurrió un error al cargar o procesar el PDF: {e}")

##### 6.3.2.3. Inspeccionar los "Chunks" o Fragmentos Finales

¡Perfecto! Esta información que me has dado es increíblemente útil y **cambia por completo el diagnóstico.**

Tu resultado demuestra que mi sospecha anterior (que el PDF era una imagen) era incorrecta. **El `PyPDFLoader` SÍ está extrayendo el texto correctamente y de forma legible.** Esta es una excelente noticia, porque significa que no necesitamos la complejidad de usar OCR.

Sin embargo, esto también nos dice que, si el texto se carga bien pero la búsqueda sigue fallando, el problema debe ocurrir en uno de los pasos intermedios:

1.  La **división del texto** (`RecursiveCharacterTextSplitter`).
2.  La **creación de los embeddings** y su almacenamiento en ChromaDB.

###### Nueva Hipótesis: "Texto Sucio" o División Inesperada

Aunque el texto es legible para nosotros, puede que contenga caracteres invisibles o un formato problemático (como saltos de línea extraños o espacios no estándar ` `) que confunden al `TextSplitter` o al modelo de `embeddings`.

Es como darle a un chef ingredientes que parecen buenos, pero que no están bien limpios. El resultado final (la búsqueda) sale mal.

###### Nuevo Test: Inspeccionar los "Chunks" o Fragmentos Finales

Vamos a realizar un nuevo diagnóstico para ver exactamente cómo queda el texto *después* de ser dividido en fragmentos (`chunks`) y justo *antes* de ser enviado a OpenAI para crear los embeddings.

**Código de Diagnóstico de Chunks:**

Este código cargará el PDF, lo limpiará (como buena práctica) y luego lo dividirá, mostrándonos los primeros 5 fragmentos resultantes.

```python
import os
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- CONFIGURACIÓN ---
PDF_PATH = "../../../assets/DG_docs/PDF_test_gradio/73SentenciaTutela.pdf"

# --- 1. FUNCIÓN DE LIMPIEZA DE TEXTO ---
# Esta es una buena práctica para normalizar el texto antes de procesarlo.
def clean_text(text: str) -> str:
    # Reemplaza el espacio no separable (un carácter común en PDFs) por un espacio normal
    text = text.replace('\xa0', ' ')
    # Reemplaza múltiples saltos de línea o espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# --- 2. CARGA Y LIMPIEZA DE DOCUMENTOS ---
print(f"📄 Cargando y limpiando el archivo: {PDF_PATH}")
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()

# Aplicamos la limpieza a cada página
for doc in pages:
    doc.page_content = clean_text(doc.page_content)

print("✅ Texto limpiado.")

# --- 3. DIVISIÓN DEL TEXTO ---
# Usamos los mismos parámetros de tu script original
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)
documents = text_splitter.split_documents(pages)
print(f"✅ Texto dividido en {len(documents)} fragmentos (chunks).")

# --- 4. INSPECCIÓN DE LOS FRAGMENTOS ---
print("\n🔍 --- Inspeccionando los primeros 5 fragmentos --- 🔍")
for i in range(min(5, len(documents))):
    print(f"\n--- FRAGMENTO {i+1} ---\n")
    print(documents[i].page_content)
    print("\n" + "="*50)

```

###### ¿Qué Debemos Buscar en el Resultado?

Ejecuta este nuevo script y presta atención a la salida de los fragmentos:

* **¿Se ve el texto coherente?** ¿O los fragmentos parecen cortados en lugares extraños?
* **¿Aparece el número de radicación `080013187005202500020-00` en alguno de los primeros fragmentos?** Debería aparecer.
* **¿Hay fragmentos muy cortos o casi vacíos?** Esto podría indicar un problema en la división.

###### El Siguiente Paso Lógico

El resultado de este script nos dará la respuesta definitiva.

* **Si los fragmentos se ven bien y coherentes,** y contienen la información clave, entonces podemos concluir con alta seguridad que el problema está en el **almacenamiento o corrupción de la base de datos ChromaDB**. La solución en ese caso sería simplemente borrar la carpeta `chroma_db` y volver a generar la base de datos desde cero con el texto ya limpio.
* **Si los fragmentos se ven mal (cortados, extraños),** entonces el problema es el `TextSplitter` y debemos ajustar sus parámetros (`chunk_size`, `chunk_overlap`, o incluso los separadores).

Estoy casi seguro de que una simple limpieza del texto y la regeneración de la base de datos resolverá el fallo. Ejecuta el script de inspección y muéstrame el resultado para dar el paso final.

###### 🪛 Código de Diagnóstico de Chunks:

In [ ]:
import os
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- CONFIGURACIÓN ---
PDF_PATH = "../../../assets/DG_docs/PDF_test_gradio/73SentenciaTutela.pdf"

# --- 1. FUNCIÓN DE LIMPIEZA DE TEXTO ---
# Esta es una buena práctica para normalizar el texto antes de procesarlo.
def clean_text(text: str) -> str:
    # Reemplaza el espacio no separable (un carácter común en PDFs) por un espacio normal
    text = text.replace('\xa0', ' ')
    # Reemplaza múltiples saltos de línea o espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# --- 2. CARGA Y LIMPIEZA DE DOCUMENTOS ---
print(f"📄 Cargando y limpiando el archivo: {PDF_PATH}")
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()

# Aplicamos la limpieza a cada página
for doc in pages:
    doc.page_content = clean_text(doc.page_content)

print("✅ Texto limpiado.")

# --- 3. DIVISIÓN DEL TEXTO ---
# Usamos los mismos parámetros de tu script original
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)
documents = text_splitter.split_documents(pages)
print(f"✅ Texto dividido en {len(documents)} fragmentos (chunks).")

# --- 4. INSPECCIÓN DE LOS FRAGMENTOS ---
print("\n🔍 --- Inspeccionando los primeros 5 fragmentos --- 🔍")
for i in range(min(5, len(documents))):
    print(f"\n--- FRAGMENTO {i+1} ---\n")
    print(documents[i].page_content)
    print("\n" + "="*50)

##### 6.3.2.4. El problema reside en la base de datos vectorial (chroma_db)

¡Excelente! Este resultado es exactamente lo que necesitábamos. Es el último paso del diagnóstico y nos lleva directamente a la causa raíz y a la solución final.

###### Análisis Final del Diagnóstico

1.  **Carga de PDF:** Confirmado, funciona.
2.  **Extracción de Texto:** Confirmado, el texto es legible y correcto.
3.  **Limpieza de Texto:** Confirmado, el texto se normaliza correctamente.
4.  **División en Chunks:** **Confirmado por tu último resultado.** Los fragmentos que me muestras son coherentes, limpios y, lo más importante, **contienen la información que buscamos**. Por ejemplo, el `FRAGMENTO 1` incluye claramente el número de Radicación.

Esto nos deja con una sola conclusión posible.

###### Causa Definitiva del Problema

El problema no está en cómo cargas, limpias o divides el texto. El problema reside en la **base de datos vectorial (`chroma_db`) que estás utilizando**.

Lo más probable es que la base de datos que tienes en tu disco se generó en un intento anterior con el texto "sucio" o mal extraído. Aunque hayas corregido el código de carga, el `retriever` sigue consultando esa base de datos antigua y corrupta, y por eso nunca encuentra nada.

###### La Solución Final: Regenerar la Base de Datos

La solución es simple y directa: debemos **borrar la base de datos antigua y crear una completamente nueva** con el proceso de carga y limpieza que ya hemos validado.

A continuación, te proporciono el **script completo y definitivo**. Este código integra todos los pasos que hemos confirmado que funcionan: la limpieza, la división y la creación de la base de datos.

**Instrucciones:**

1.  **MUY IMPORTANTE:** Ve a la carpeta de tu proyecto y **elimina manualmente la carpeta `chroma_db`**.
2.  Ejecuta el siguiente script. Este creará una nueva carpeta `chroma_db` poblada con la información limpia y correcta.

####### Script Consolidado para Regenerar la Base de Datos

```python
import os
import re
import shutil
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# --- CONFIGURACIÓN ---
PDF_DIRECTORY = "../../../assets/DG_docs/PDF_test_gradio/"
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small"  # Usamos un modelo más moderno y eficiente

print("--- Iniciando el Proceso Completo de Creación de la Base de Datos Vectorial ---")

# 1. Asegúrate de que la clave API esté configurada
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

# 2. Eliminar la base de datos antigua si existe (por seguridad)
if os.path.exists(PERSIST_DIRECTORY):
    print(f"🗑️ Eliminando la base de datos antigua en '{PERSIST_DIRECTORY}'...")
    shutil.rmtree(PERSIST_DIRECTORY)
    print("✅ Base de datos antigua eliminada.")

# 3. Función de limpieza de texto
def clean_text(text: str) -> str:
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 4. Cargar y procesar todos los PDFs del directorio
all_pages = []
pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.lower().endswith(".pdf")]

print(f"\n📄 Encontrados {len(pdf_files)} archivos PDF para procesar...")

for pdf_file in pdf_files:
    pdf_path = os.path.join(PDF_DIRECTORY, pdf_file)
    print(f"  -> Cargando y limpiando '{pdf_file}'...")
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    for doc in pages:
        doc.page_content = clean_text(doc.page_content)
    all_pages.extend(pages)

print(f"\n✅ Total de páginas cargadas y limpiadas: {len(all_pages)}")

# 5. Dividir los documentos en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)
documents = text_splitter.split_documents(all_pages)
print(f"✅ Texto dividido en {len(documents)} fragmentos (chunks).")


# 6. Crear embeddings y la base de datos vectorial
print(f"\n🧠 Creando embeddings con el modelo '{EMBEDDING_MODEL}' y guardando en ChromaDB...")
print("(Esto puede tardar unos minutos y consumir tokens de OpenAI...)")

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=api_key_environ)

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=PERSIST_DIRECTORY
)

print("\n🎉 ¡ÉXITO! Tu nueva base de datos vectorial ha sido creada y guardada correctamente.")
print(f"Ahora puedes usar el directorio '{PERSIST_DIRECTORY}' para tus consultas.")

```

###### Siguientes Pasos

Una vez que este script termine de ejecutarse, tu sistema estará listo.

Vuelve a ejecutar el **primer script de test que te di** (el que tenía los 3 casos de prueba). Ahora, el resultado debería ser:

`✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.`

Cuando eso suceda, habremos solucionado el problema de raíz y podrás conectar tu `retriever` al resto de tu aplicación con la confianza de que está encontrando la información correcta.

###### 🪛 Script Consolidado para Regenerar la Base de Datos

In [ ]:
import os
import re
import shutil
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# --- CONFIGURACIÓN ---
PDF_DIRECTORY = "../../../assets/DG_docs/PDF_test_gradio/"
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small"

print("--- Iniciando el Proceso Completo de Creación de la Base de Datos Vectorial ---")

# 1. Asegúrate de que la clave API esté configurada
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

# 2. Eliminar la base de datos antigua si existe
if os.path.exists(PERSIST_DIRECTORY):
    print(f"🗑️ Eliminando la base de datos antigua en '{PERSIST_DIRECTORY}'...")
    shutil.rmtree(PERSIST_DIRECTORY)
    print("✅ Base de datos antigua eliminada.")

# 3. Función de limpieza de texto
def clean_text(text: str) -> str:
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# 4. Cargar y procesar todos los PDFs
all_pages = []
pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.lower().endswith(".pdf")]
print(f"\n📄 Encontrados {len(pdf_files)} archivos PDF para procesar...")
for pdf_file in pdf_files:
    pdf_path = os.path.join(PDF_DIRECTORY, pdf_file)
    try:
        print(f"  -> Cargando y limpiando '{pdf_file}'...")
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        for doc in pages:
            doc.page_content = clean_text(doc.page_content)
            # Filtramos páginas que quedaron vacías después de limpiar
            if doc.page_content:
                all_pages.append(doc)
    except Exception as e:
        print(f"  ⚠️ Error al procesar el archivo {pdf_file}: {e}. Omitiendo.")

print(f"\n✅ Total de páginas cargadas y limpiadas: {len(all_pages)}")

# 5. Dividir los documentos en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
)
documents = text_splitter.split_documents(all_pages)
print(f"✅ Texto dividido en {len(documents)} fragmentos (chunks).")


# --- 6. (MODIFICADO) Crear embeddings y la base de datos POR LOTES ---
print(f"\n🧠 Creando embeddings con el modelo '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL, openai_api_key=api_key_environ)

print(f"✅ Inicializando base de datos vacía en '{PERSIST_DIRECTORY}'...")
# Inicializamos la base de datos vacía
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY, 
    embedding_function=embeddings
)

# Definimos el tamaño del lote (un número seguro para no exceder los límites)
batch_size = 200
total_docs = len(documents)
print(f"\n➕ Añadiendo {total_docs} documentos a la base de datos en lotes de {batch_size}...")

for i in range(0, total_docs, batch_size):
    # Obtenemos el lote actual de documentos
    batch = documents[i:i + batch_size]
    
    # Calculamos el número de lote actual para mostrar el progreso
    batch_number = (i // batch_size) + 1
    total_batches = (total_docs + batch_size - 1) // batch_size
    
    print(f"  -> Procesando lote {batch_number}/{total_batches} (documentos {i+1} a {min(i + batch_size, total_docs)})...")
    
    # Añadimos el lote a la base de datos. La llamada a la API de OpenAI ocurre aquí.
    vectorstore.add_documents(batch)

print("\n🎉 ¡ÉXITO! Tu nueva base de datos vectorial ha sido creada y guardada correctamente.")
print(f"Ahora puedes usar el directorio '{PERSIST_DIRECTORY}' para tus consultas.")

##### 6.3.2.5. Objetivo del Test: Evaluar `de nuevo` al Retriever. Tres preguntas del PDF "73SentenciaTutela.pdf" 

###### Objetivo del Test: Evaluar al `Retriever`

El `retriever` es el componente encargado de buscar en tu base de datos vectorial (ChromaDB) los fragmentos de texto más relevantes para la pregunta que haces. Si no le entrega los fragmentos correctos al modelo de lenguaje (LLM), es imposible que el LLM genere una buena respuesta.

Vamos a ejecutar el `retriever` de forma aislada para inspeccionar su "trabajo".

---

###### Paso 1: Escoge un Caso de Prueba Específico

Primero, necesitas un "caso de oro" que puedas verificar manualmente.

1.  **Busca en tus PDFs:** Abre uno de los documentos PDF que usaste para crear la base de datos. Encuentra un dato muy específico y único. Por ejemplo, el nombre de un proyecto, un número de contrato, una fecha clave o una frase técnica.
2.  **Define tu "Golden Pair":**
    * **Pregunta de prueba:** Formula una pregunta que *debería* encontrar ese dato. Ejemplo: `"¿Cuál es el número del contrato para el mantenimiento de la vía Ocaña-Cúcuta?"`
    * **Contenido esperado:** Anota la frase o las palabras clave que *obligatoriamente* deben estar en los fragmentos recuperados para poder responder la pregunta. Ejemplo: `"contrato de obra pública No. 2345-2023"`.

###### Paso 2: Código para Ejecutar el Test

He preparado una versión simplificada de tu script que se enfoca **únicamente en realizar este test**. No utilizaremos el LLM para generar respuestas, solo para cargar los embeddings y usar el `retriever`.

Copia y ejecuta el siguiente código. Asegúrate de modificar las variables de la sección "CONFIGURACIÓN DEL TEST".

```python
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "que pasa en Ocaña?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["Ocaña", "emergencia", "deslizamiento"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

```

###### ¿Cómo Interpretar los Resultados?

1.  **Si el veredicto es "ÉXITO"**: Significa que tu base de datos y tu `retriever` funcionan correctamente. El problema podría estar en el `prompt` que le das al LLM o en el propio LLM (aunque es menos probable).
2.  **Si el veredicto es "FALLO"**: Confirma tu sospecha. El problema está en la base de conocimiento. El sistema no es capaz de encontrar los fragmentos de texto relevantes.

###### ¿Qué Hacer si el Test Falla?

Si el test falla, aquí tienes las causas más probables y cómo solucionarlas:

1.  **Problema con la División de Texto (`chunking`)**:
    * **Causa**: El `chunk_size=1500` puede ser muy grande, y la información clave se "diluye" con texto irrelevante en el mismo fragmento. O puede ser muy pequeño y parte una frase importante.
    * **Solución**: Prueba a regenerar tu base de datos con un `chunk_size` más pequeño (ej. 1000) y un `chunk_overlap` mayor (ej. 250). Esto crea más fragmentos, pero más enfocados.

2.  **Problema con la Extracción de Texto del PDF**:
    * **Causa**: Algunos PDFs son en realidad imágenes de texto (escaneados). `PyPDFLoader` no puede leer texto de imágenes.
    * **Solución**: Verifica el contenido de tus documentos justo después de cargarlos (la variable `ml_papers` en tu script original). Si ves caracteres extraños o texto sin sentido, necesitas usar una herramienta de **OCR (Reconocimiento Óptico de Caracteres)** para extraer el texto de esos PDFs antes de pasarlos a LangChain.

3.  **Problema con el Modelo de Embeddings**:
    * **Causa**: `text-embedding-ada-002` es un modelo más antiguo. Puede que no esté capturando el significado semántico de tu pregunta con la suficiente precisión.
    * **Solución**: Regenera tu base de datos usando un modelo más moderno y potente como `"text-embedding-3-small"` (más barato y eficiente) o `"text-embedding-3-large"` (máxima calidad). Tu propio código ya tiene un comentario sugiriendo este cambio.

Ejecuta este test y, según el resultado, podremos enfocarnos en la solución correcta.

###### 🔁 PREGUNTA 1: ¿Cuál es el número de radicación de la acción de tutela de Gersel Luis Pérez Altamiranda?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Cuál es el número de radicación de la acción de tutela de Gersel Luis Pérez Altamiranda?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["080013187005202500020-00"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

###### 🔁 PREGUNTA 2: ¿Qué resolución distribuye el cobro de la Contribución Nacional de Valorización para el proyecto Circunvalar de la Prosperidad?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Qué resolución distribuye el cobro de la Contribución Nacional de Valorización para el proyecto Circunvalar de la Prosperidad?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["Resolución No. 3856", "distribuye el cobro"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

###### 🔁 PREGUNTA 3: ¿Bajo qué condición están excluidas del cobro de valorización las tierras de grupos étnicos?

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Bajo qué condición están excluidas del cobro de valorización las tierras de grupos étnicos?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["excluidos", "tierras comunales", "grupos étnicos", "titularidad radique en una entidad de derecho público"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

##### 6.3.2.6. Objetivo del Test 2: Evaluar `de nuevo` al Retriever. Tres preguntas del PDF "2024S-VBOG-059529.pdf" 

###### 🔁 PREGUNTA 1: 
- Pregunta de Prueba (TEST_QUERY): ¿Cuál es el valor actualizado del contrato de obra 0987 de 2021?

- Contenido Esperado (EXPECTED_KEYWORDS): ["0987 DE 2021", "Valor actualizado", "324,627,245,375.00"]


In [6]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Cuál es el valor actualizado del contrato de obra 0987 de 2021?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["0987 DE 2021", "Valor actualizado", "324,627,245,375.00"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

🧪 Iniciando el Test de Recuperación de Contexto...
Buscando en la base de datos en: 'chroma_db'
Cargando embeddings con el modelo: 'text-embedding-3-small'...

🔍 Ejecutando retriever con la pregunta: '¿Cuál es el valor actualizado del contrato de obra 0987 de 2021?'

📄 --- Fragmentos Recuperados --- 📄

--- Documento 1 (Fuente: ../../../assets/DG_docs/PDF_test_gradio/2024S-VBOG-082247.pdf) ---

CONTRATO 964 DE 2021 VIGENCIA VALOR 2023 $27.072.000.000 2024 $54.144.000.000 2025 $ 63.168.000.000 2026 $ 72.192.000.000 2027 $ 67.680.000.000 2028 $ 63.168.000.000 2029 $ 54.144.000.000 2030 $ 94.752.000.000 Total $ 541.389.953.817 CONTRATO 1111 DE 2021 VIGENCIA VALOR 2021 $30.402.106.819 2022 $20.379.255.249 2023 $29.328.000.000 2024 $61.056.000.000 2025 $ 71.232.000.000 2026 $81.408.000.000 2027 $ 76.320.000.000 2028 $ 71.232.000.000 2029 $ 61.656.000.000 2030 $ 107.448.000.000 Total $ 610.461.362.068 De igual forma, se relacionan las modificaciones adelantadas en el marco de los contratos 96

Created dataset file at: .gradio\flagged\dataset1.csv


###### 🔁 PREGUNTA 2: 

- Pregunta de Prueba (Test Query): ¿Qué inversión ha realizado INVÍAS para cumplir la acción popular en el corredor Curos-Málaga?

- Contenido Esperado (Expected Keywords): ["acción popular", "68001-23-33-000-2015-00847", "invierte", "316.664.639.074", "contrato de obra No. 1042 de 2021"]

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿Qué inversión ha realizado INVÍAS para cumplir la acción popular en el corredor Curos-Málaga?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["acción popular", "68001-23-33-000-2015-00847", "invierte", "316.664.639.074", "contrato de obra No. 1042 de 2021"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

###### 🔁 PREGUNTA 3: 

Pregunta de Prueba (Test Query): ¿Cuál es el avance físico y financiero del proyecto de la Variante de San Gil?

Contenido Esperado (Expected Keywords): ["Variante de San Gil", "avance físico", "96.69%", "avance financiero", "95.19%"]

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# --- CONFIGURACIÓN DEL TEST ---
# 1. Asegúrate de que tu clave de API esté configurada como variable de entorno
#    os.environ["OPENAI_API_KEY"] = "tu_clave_aqui"

# 2. Define la pregunta y el contenido que esperas encontrar
TEST_QUERY = "¿¿Cuál es el avance físico y financiero del proyecto de la Variante de San Gil?"  # <--- CAMBIA ESTO por tu pregunta de prueba
EXPECTED_KEYWORDS = ["Variante de San Gil", "avance físico", "96.69%", "avance financiero", "95.19%"] # <--- CAMBIA ESTO por las palabras clave que deben aparecer

# 3. Define la ruta a tu base de datos vectorial
PERSIST_DIRECTORY = "chroma_db"
EMBEDDING_MODEL = "text-embedding-3-small" # Usa el mismo modelo con el que creaste la base

# --- LÓGICA DEL TEST ---

print("🧪 Iniciando el Test de Recuperación de Contexto...")

# 1. Validar configuración
api_key_environ = os.environ.get("OPENAI_API_KEY")
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada.")

if not os.path.exists(PERSIST_DIRECTORY):
    raise FileNotFoundError(f"El directorio de la base de datos '{PERSIST_DIRECTORY}' no existe. Asegúrate de haber ejecutado el script principal primero.")

print(f"Buscando en la base de datos en: '{PERSIST_DIRECTORY}'")

# 2. Cargar el retriever
print(f"Cargando embeddings con el modelo: '{EMBEDDING_MODEL}'...")
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectorstore = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
# Aumentamos 'k' para tener más contexto que inspeccionar
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 3. Ejecutar el retriever de forma aislada
print(f"\n🔍 Ejecutando retriever con la pregunta: '{TEST_QUERY}'")
retrieved_docs = retriever.invoke(TEST_QUERY)

# 4. Analizar los resultados
print("\n📄 --- Fragmentos Recuperados --- 📄")
if not retrieved_docs:
    print("❌ ¡ERROR! El retriever no devolvió ningún documento.")
else:
    for i, doc in enumerate(retrieved_docs):
        source = doc.metadata.get("source", "desconocido")
        content = doc.page_content
        print(f"\n--- Documento {i+1} (Fuente: {source}) ---\n")
        print(content)
        print("-" * 40)

# 5. Verificación automática
print("\n🏁 --- Verificación de Palabras Clave --- 🏁")
found_keywords_count = 0
for doc in retrieved_docs:
    for keyword in EXPECTED_KEYWORDS:
        if keyword.lower() in doc.page_content.lower():
            print(f"✅ Palabra clave encontrada: '{keyword}' en Fuente: {doc.metadata.get('source', 'N/A')}")
            found_keywords_count += 1

print("\n--- Veredicto del Test ---")
if found_keywords_count > 0:
    print("✅ ÉXITO PARCIAL/TOTAL: Se encontraron uno o más de los contenidos esperados en los fragmentos recuperados.")
else:
    print("❌ FALLO: Ninguna de las palabras clave esperadas fue encontrada. El contexto relevante no se está recuperando.")

### 6.4. Con memoria – El sistema conserva el contexto

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0),
    retriever=vectorstore.as_retriever(),
    memory=memory
)

# Simulación de una conversación con contexto
chat_history = []

# Primera pregunta
response_1 = qa_chain({"question": "¿Qué entidad está a cargo del contrato de la vía Ocaña-Cúcuta?"})
print("Respuesta 1:", response_1['answer'])

# Segunda pregunta que se apoya en la anterior
response_2 = qa_chain({"question": "¿Cuál es el plazo de entrega del contrato?"})
print("Respuesta 2:", response_2['answer'])
# Aquí, la segunda pregunta sí aprovecha el contexto anterior, gracias a la memoria.

## 7. LangChain para INVIAS


### 7.1. Descripción del código

El archivo `langc_v01.ipynb` es un **notebook de Jupyter** diseñado para mostrar paso a paso cómo construir un **sistema de consulta de información basado en documentos PDF usando LangChain y modelos de lenguaje de OpenAI**. 

#### Propósito del archivo `langc_v01.ipynb`

Implementar un **chatbot inteligente** que pueda:

1. **Leer documentos PDF** institucionales.
2. **Convertirlos en embeddings semánticos** usando OpenAI.
3. **Almacenarlos en una base vectorial persistente** (Chroma).
4. **Responder preguntas** formuladas en lenguaje natural usando LLMs.

#### Estructura del notebook

1. **Carga y lectura de documentos PDF**

   * Usa loaders como `PyPDFLoader`.
   * Fragmenta el contenido en chunks con metadatos.
   * Prepara los datos para el embedding.

2. **Generación de embeddings y base vectorial**

   * Usa `OpenAIEmbeddings` (`text-embedding-3-small`).
   * Crea y guarda una base en Chroma (`chroma_db`).
   * Permite que la búsqueda de contexto sea semántica, no solo por palabras clave.

3. **Creación del `retriever`**

   * Define cuántos documentos relevantes recuperar (`k=3` o `k=5`).
   * Es el núcleo del sistema RAG (Retrieval-Augmented Generation).

4. **Construcción de la cadena de QA (`RetrievalQA`)**

   * Conecta el `retriever` con un modelo de lenguaje (`ChatOpenAI`).
   * Puede configurarse para devolver solo la respuesta, o también las fuentes.

5. **Ejecución de consultas**

   * Envía preguntas como `"¿Qué es CCPT?"` o `"¿Qué acciones tomó el INVÍAS en El Tarrita?"`.
   * El sistema responde basándose en los documentos cargados.

> Este codigo viene de [langc_v01.ipynb](../../../Platzi_codes/langc/v01/langc_v01.ipynb). Este código desarrollo tiene [Code Explanation](../../../AI_Queries/code_explanation/).  

### 7.2. Test del código

- [QA_01.md](others/QA_01.md)
- [QA_02.md](others/QA_02.md)
- [QA_03.md](others/QA_03.md)

### 7.3. Hallazgos 🚩

1. No cita las fuentes de donde extrae la información.
2. Solo para una pregunta. Si uso LangChain con un `ConversationalRetrievalChain`, se puede hacer varias preguntas en la misma sesión
3. Sin memoria – Cada pregunta se responde de forma aislada